# Naïve Bayes Classifier

## Define nodes and edges

We first need to build a list of nodes and edges for the graph given the data in the data file. We define two function to do so: `create_nodes_from_header()` and `create_edges_from_nodes()`.

In [55]:
import csv

def create_nodes_from_header(filename):
    """
        Returns a list of node labels for an NBC from the CSV file at *filename*.
        Assumes that the first row of this file contains unquoted node labels.
    """
    csv_file = open(filename, 'r')
    reader = csv.reader(csv_file)
    nodes = reader.next()
    csv_file.close()
    return nodes

def create_edges_from_nodes(nodes):
    """
        Returns a list of edges for an NBC given the list of nodes in *nodes*.
        Assumes that the first node in *nodes* is the class, and all others are
        features.
    """
    class_label = nodes[0]
    return [[class_label, feature] for feature in nodes[1:]]

data_file_path = 'mushroom.csv'

nodes = create_nodes_from_header(data_file_path)
edges = create_edges_from_nodes(nodes)

from pprint import pprint

print('Nodes:')
pprint(nodes)

print('Edges:')
pprint(edges)

Nodes:
['poisonous',
 'cap_shape',
 'cap_surface',
 'cap_color',
 'bruises',
 'odor',
 'gill_attachment',
 'gill_spacing',
 'gill_size',
 'gill_color',
 'stalk_shape',
 'stalk_root',
 'stalk_surface_above_ring',
 'stalk_surface_below_ring',
 'stalk_color_above_ring',
 'stalk_color_below_ring',
 'veil_type',
 'veil_color',
 'ring_number',
 'ring_type',
 'spore_print_color',
 'population',
 'habitat']
Edges:
[['poisonous', 'cap_shape'],
 ['poisonous', 'cap_surface'],
 ['poisonous', 'cap_color'],
 ['poisonous', 'bruises'],
 ['poisonous', 'odor'],
 ['poisonous', 'gill_attachment'],
 ['poisonous', 'gill_spacing'],
 ['poisonous', 'gill_size'],
 ['poisonous', 'gill_color'],
 ['poisonous', 'stalk_shape'],
 ['poisonous', 'stalk_root'],
 ['poisonous', 'stalk_surface_above_ring'],
 ['poisonous', 'stalk_surface_below_ring'],
 ['poisonous', 'stalk_color_above_ring'],
 ['poisonous', 'stalk_color_below_ring'],
 ['poisonous', 'veil_type'],
 ['poisonous', 'veil_color'],
 ['poisonous', 'ring_number'],
 

## Parse observations

Now, we parse observations from the data file with a new function, `create_observations_from_csv()`.

In [56]:
def create_observations_from_csv(filename):
    """
        Returns a dictionary of observations from data in the CSV file at *filename*.
        Assumes that the first row of this file contains node labels.
    """
    observations = []
    csv_file = open(filename)
    reader = csv.DictReader(csv_file, fieldnames=nodes)
    reader.next()
    for row in reader:
        observation = dict()
        for node in nodes:
            observation[node] = row[node]
        observations.append(observation)
    return observations

observations = create_observations_from_csv(data_file_path)
pprint(observations[0])

{'bruises': 't',
 'cap_color': 'n',
 'cap_shape': 'x',
 'cap_surface': 's',
 'gill_attachment': 'f',
 'gill_color': 'k',
 'gill_size': 'n',
 'gill_spacing': 'c',
 'habitat': 'u',
 'odor': 'p',
 'poisonous': 'p',
 'population': 's',
 'ring_number': 'o',
 'ring_type': 'p',
 'spore_print_color': 'k',
 'stalk_color_above_ring': 'w',
 'stalk_color_below_ring': 'w',
 'stalk_root': 'e',
 'stalk_shape': 'e',
 'stalk_surface_above_ring': 's',
 'stalk_surface_below_ring': 's',
 'veil_color': 'w',
 'veil_type': 'p'}


## Building the graph skeleton and node data

`libpgm` uses the `GraphSkeleton` and `NodeData` classes to represent the nodes/edge and details about the nodes, respectively. We build a bare `GraphSkeleton` using `nodes` and `edges` and order the `GraphSkeleton` in a topological ordering with respect to the edges in edges using `GraphSkeleton.toporder()`. When we learn the model parameters, a `NodeData` object will be built for us.

In [57]:
from libpgm.graphskeleton import GraphSkeleton

def create_graph_skeleton(nodes, edges):
    graphSkeleton = GraphSkeleton()
    graphSkeleton.V = nodes
    graphSkeleton.E = edges
    graphSkeleton.toporder()
    return graphSkeleton

graphSkeleton = create_graph_skeleton(nodes, edges)

## Training the model

`libpgm` provides the method `pgmLearner.discrete_mle_estimateparams()` for estimating the parameters of a `DiscreteBayesianNetwork` given a set of observations. We use this in a `train_model()` function in order to estimate the parameters of our Bayesian network.

In [58]:
from libpgm.pgmlearner import PGMLearner

def train_model(skeleton, observations):
    learner = PGMLearner()
    bn = learner.discrete_mle_estimateparams(skeleton, observations)
    return bn.V, bn.E, bn.Vdata

To test this method, we train the model on our entire dataset. This method returns a list of nodes, a list of edges, and a node data dictionary. We've specified the nodes and edges already, but we'll need the parameters `discrete_mle_estimateparams` gives us in the node data dictionary.

In [59]:
learned_nodes, learned_edges, learned_node_data = train_model(graphSkeleton, observations)
pprint(learned_node_data)

{'bruises': {'children': [],
             'cprob': {"['e']": [0.6539923954372624, 0.34600760456273766],
                       "['p']": [0.15934627170582227, 0.8406537282941777]},
             'numoutcomes': 2,
             'parents': ['poisonous'],
             'vals': ['t', 'f']},
 'cap_color': {'children': [],
               'cprob': {"['e']": [0.30038022813688214,
                                   0.09505703422053231,
                                   0.17110266159695817,
                                   0.24524714828897337,
                                   0.1482889733840304,
                                   0.013307984790874524,
                                   0.011406844106463879,
                                   0.0038022813688212928,
                                   0.0076045627376425855,
                                   0.0038022813688212928],
                         "['p']": [0.2604698672114402,
                                   0.17160367722165476,
      

## Performing inference

We can now perform inference using our network. `libpgm`'s `TableCPDFactorization` provides methods for exact inference. We'll use the `specificquery()` method for doing so. `libpgm` is strange in the way it manipulates the network and its factors when performing inference. To work around this, we define two helper functions: `pristine_bn()` and `pristine_fn`. `pristine_bn()` takes the nodes, edges, and node data that we've already learned and creates a fresh network. With this network, `pristine_fn()` returns a new `TableCPDFactorization` that we can use for another query. Note that returned probabilities from `specificquery()` are normalized.

In [60]:
from libpgm.discretebayesiannetwork import DiscreteBayesianNetwork
from libpgm.tablecpdfactorization import TableCPDFactorization

def pristine_bn(V, E, Vdata):
    fresh_bn = DiscreteBayesianNetwork()
    fresh_bn.V = list(V)
    fresh_bn.E = list(E)
    fresh_bn.Vdata = Vdata.copy()
    return fresh_bn

def pristine_fn(V, E, Vdata):
    pristine = pristine_bn(V, E, Vdata)
    return TableCPDFactorization(pristine)

sample_query = dict(poisonous='p')
sample_evidence = dict(habitat='u')
result = pristine_fn(learned_nodes, learned_edges, learned_node_data).specificquery(sample_query, sample_evidence)
print('P(poisonous=p|habitat=u): {0}'.format(result))

P(poisonous=p|habitat=u): 0.739130434783


## Classifying individual observations

Now that we can perform inference, we can classify observations. We have a function, `classify_observation` that does just this.

There are two important things to node about `classify_observation()`. First, `remove_missing_data()` is called for each observation. If a given observation is missing a value for a particular variable, `remove_missing_data()` removes this variable from the observation (and, in turn, the evidence used in inference) altogether. Second, while this will not occur when the network parameters have been trained on the entire dataset, it may be the case that the value for a variable in a given observation was not seen in the training data. When this occurs, we must remove this variable from the evidence as well--we do so in `remove_untrained_values()`.

In [61]:
def classify_observation(observation, V, E, Vdata):
    o = observation.copy()
    remove_missing_data(o)
    remove_untrained_values(o, Vdata)

    if 'poisonous' not in o.keys():
        return None

    actual_class = o['poisonous']
    del o['poisonous']

    map_class = None
    map_prob = -1
    try:
        for value in Vdata['poisonous']['vals']:
            query = dict(poisonous=value)
            evidence = dict(o)
            factorization = pristine_fn(V, E, Vdata)
            result = factorization.specificquery(query, evidence)
            if result > map_prob:
                map_class = value
                map_prob = result
        return map_class
    except:
        return None

def remove_missing_data(observation):
    for key in observation.keys():
        if observation[key] == '?':
            del observation[key]

def remove_untrained_values(observation, learned_Vdata):
    for key in observation.keys():
        observation_value = observation[key]
        known_values = learned_Vdata[key]['vals']
        if observation_value not in known_values:
            del observation[key]

result = classify_observation(observations[42], learned_nodes, learned_edges, learned_node_data)
print('Estimated value of \'poisonous\' for observations[42]: {0}'.format(result))
print('Actual value of \'poisonous\' for observations[42]: {0}'.format(observations[42]['poisonous']))

Estimated value of 'poisonous' for observations[42]: e
Actual value of 'poisonous' for observations[42]: e


## Classifying groups of observations

Finally, we define `test_model` for testing the accuracy of the model given a group of observations. This function passes each observation in turn to `classify_observation` and returns the number of correct classifications and the total number of classifications.

In [65]:
def test_model(observations, V, E, Vdata):
    correct = 0
    for observation in observations:
        map_class = classify_observation(observation, V, E, Vdata)
        if map_class == observation['poisonous']:
            correct += 1

    return correct, len(observations)

correct, total = test_model(observations[0:10], learned_nodes, learned_edges, learned_node_data)
print('Correct: {0}, Total: {1}'.format(correct, total))

Correct: 10, Total: 10


## Creating indices for k-fold cross validation

We'll need a way to create indices for indexing into our list of observations in order to split them into test and training sets for k-fold cross validation. The helper method `k_fold_indices()` does just this.

In [62]:
def k_fold_indices(k, n):
    testing_size = n / k
    training_size = n - testing_size

    validation_sets = []
    all_indices = range(n)    
    for i in range(k):
        training_indices = list(all_indices)
        index = testing_size * i
        testing_indices = range(index,index + testing_size)
        for j in testing_indices:
            training_indices.remove(j)
        validation_sets.append([training_indices, testing_indices])
    return validation_sets

pprint(k_fold_indices(5, 10))

[[[2, 3, 4, 5, 6, 7, 8, 9], [0, 1]],
 [[0, 1, 4, 5, 6, 7, 8, 9], [2, 3]],
 [[0, 1, 2, 3, 6, 7, 8, 9], [4, 5]],
 [[0, 1, 2, 3, 4, 5, 8, 9], [6, 7]],
 [[0, 1, 2, 3, 4, 5, 6, 7], [8, 9]]]


## Performing k-fold cross validation

Finally, we use `run_cross_validation()` to perform k-fold cross validation of our model. Using the indices obtained from `k_fold_indices()`, `run_cross_validation()` partitions the dataset into training and testing sets, uses `test_model()` to calculate the accuracy for each validation run, and reports the average accuracy across all validation runs.

In [66]:
def run_cross_validation(k, observations, graphSkeleton):
    cv_indices = k_fold_indices(k, len(observations))

    print('Performing {0}-fold cross validation.'.format(k))

    accuracies = []

    for i in range(len(cv_indices)):
        print('\tRunning iteration {0}.'.format(i + 1))
        testing = [observations[j] for j in cv_indices[i][1]]
        training = [observations[j] for j in cv_indices[i][0]]

        learned_V, learned_E, learned_Vdata = train_model(graphSkeleton, training)

        correct, n = test_model(testing, learned_V, learned_E, learned_Vdata)
        accuracy = correct/float(n)
        accuracies.append(accuracy)

        print('\t\tAccuracy: {0}'.format(accuracy))

    print('\tWeighted accuracy: {0}'.format(sum(accuracies)/len(accuracies)))

run_cross_validation(10, observations, graphSkeleton)

Performing 10-fold cross validation.
	Running iteration 1.
		Accuracy: 1.0
	Running iteration 2.
		Accuracy: 1.0
	Running iteration 3.
		Accuracy: 1.0
	Running iteration 4.
		Accuracy: 1.0
	Running iteration 5.
		Accuracy: 1.0
	Running iteration 6.
		Accuracy: 0.996305418719
	Running iteration 7.
		Accuracy: 0.971674876847
	Running iteration 8.
		Accuracy: 0.996305418719
	Running iteration 9.
		Accuracy: 0.995073891626
	Running iteration 10.
		Accuracy: 0.983990147783
	Weighted accuracy: 0.994334975369
